In [ ]:
import json
import bz2
import os
import csv
import datetime
from bz2 import BZ2File
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#path to data, json and our corpus
pathToData = "data/"
pathToJson = "jsonl_created/" 
pathToBz2 = "ourCorpusBZ2/"
files_to_read = os.listdir(pathToData) 

In [ ]:
# a helper function to get the JSON lines from an archive 
def read_jsonlines(bz2_file):
    text = bz2_file.read().decode('utf-8')
    for line in text.split('\n'):
        if line != '':
            yield line

In [ ]:
#This function clean the whole corpus into a smaller corpus with the list of our words (not case sensitive)
#It creates another corpus of the format bz2 and it keeps the json format.
def createOurCorpus():
    nb =1
    count =0
    returntxt = open("txtname.txt", "w+")
    for file in files_to_read:
        filename = file[0:14]
        with open("jsonl_created/"+filename, 'w') as outfile:  
            text_article = ""
            #open the file 
            f = BZ2File(os.path.join(pathToData, file), 'r')
            # get the list of articles it contains (= a json object on each line)
            articles = list(read_jsonlines(f))
            # load the first 100 articles as json and access their attributes
            for a in articles:
                # decode the json string into an object (dict)
                json_article = json.loads(a)
                jsonString = {}
                ### Change here the keyword to search for
                if "apartheid" or "afrique du sud" or "sharpeville" or "ssca" or "anc" or "sarb" or "soweto" or "mandela" or "pretoria" or "johannesburg" or "bloemfontein" or "le cap" in json_article["ft"].lower(): ### expression régulière pour checker ca
                    text_article = text_article + " \n**** *id:" + str(count) + " *from:" + file + "  \n " + json_article["ft"]
                    jsonString = {'id' : str(count), 'd' : json_article["d"], 'ft' : json_article["ft"], 'pp' : json_article["pp"]}
                    json.dump(jsonString, outfile)
                    outfile.write('\n')
                    count += 1
            returntxt.write(text_article)
            nb = nb + 1 
        f = bz2.compress(open(pathToJson + filename, 'rb').read())
        fh = open(pathToBz2 + filename + "A.bz2", 'wb')
        fh.write(f)
        fh.close()
    returntxt.close()

In [ ]:
createOurCorpus()

In [ ]:
#changing the path to our corpus
files_to_read = os.listdir(pathToBz2)
files_to_read

In [ ]:
#This function clean our corpus into a smaller corpus with the word apartheid (not case sensitive)
#It creates another corpus of the format bz2 and it keeps the json format.
def createCorpusApartheid():
    #From the big corpus to our corpus in json.bz2
    nb =1
    count =0
    returntxt = open("Apartheid/txtname.txt", "w+")
    for file in files_to_read:
        filename = file[0:14]
        with open("Apartheid/json/"+filename, 'w') as outfile:  
            text_article = ""
            #open the file 
            f = BZ2File(os.path.join(pathToBz2, file), 'r')
            # get the list of articles it contains (= a json object on each line)
            articles = list(read_jsonlines(f))
            # load the first 100 articles as json and access their attributes
            for a in articles:
                # decode the json string into an object (dict)
                json_article = json.loads(a)
                jsonString = {}
                ### Change here the keyword to search for (can add more than one connected with AND / OR)
                if "apartheid" in json_article["ft"].lower(): 
                    text_article = text_article + " \n**** *id:" + str(count) + " *from:" + file + "  \n " + json_article["ft"]
                    jsonString = {'id' : str(count), 'd' : json_article["d"], 'ft' : json_article["ft"], 'pp' : json_article["pp"]}
                    json.dump(jsonString, outfile)
                    outfile.write('\n')
                    count += 1
            returntxt.write(text_article)
            nb = nb + 1 
        f = bz2.compress(open("Apartheid/json/" + filename, 'rb').read())
        fh = open("Apartheid/bz2/" + filename + "A.bz2", 'wb')
        fh.write(f)
        fh.close()
    returntxt.close()

In [ ]:
createCorpusApartheid()

In [ ]:
files_to_read = os.listdir("Apartheid/bz2/")
files_to_read

In [ ]:
#This function selects the date in our corpus that are between the 2 delta date. 
# y: year, m: month, d: day of the event deltap: the delta from date of the event to max in days, deltan: the delta to date of the event from min in days
def csvtxtwriter(csvname, y, m, d, deltap, deltan):
    nb =0
    count =0
    dateMax = datetime.timedelta(days = deltap)
    dateMin = datetime.timedelta(days = deltan)
    dateEventstring = str(y)+"-"+str(m) + "-"+str(d)
    dateEvent = datetime.datetime.strptime(dateEventstring, '%Y-%m-%d')
    #path to the csv, if "Apartheid" then we look at the small corpus with only the word apartheid
    pathCSV = "Apartheid/csv/"
    text_articletxt = ""
    #open the csv to write inside
    with open(pathCSV + csvname, 'w', newline='') as csvFinal:
        linewriter = csv.writer(csvFinal, delimiter = '*')
        linewriter.writerow(["id", "date"])
        for file in files_to_read:
            nb +=1
            #open the file 
            f = BZ2File(os.path.join("Apartheid/bz2/", file), 'r') #if the path doesnt have the Apartheid/ it looks at our corpus with a list of words
            # get the list of articles it contains (= a json object on each line)
            articles = list(read_jsonlines(f))
            # load the first 100 articles as json and access their attributes
            for a in articles:
                # decode the json string into an object (dict)
                json_article = json.loads(a)
                datearticle = datetime.datetime.strptime(json_article["d"], '%Y-%m-%d')
                if ((datearticle - dateEvent < dateMax) and (datearticle - dateEvent > dateMin)) :
                    #write the date
                    linewriter.writerow([str(count), json_article["d"][:7]]) ###if we want a groupbyday: remove "[:7]"
                    count += 1

In [ ]:
#The date we want: year / month / day 
datey = [1948, 1960, 1976, 1983, 1986, 1990, 1990, 1962, 1949]
datem = [6, 3, 6, 5, 6, 3, 2, 8, 6]
dated = [15, 21, 16, 20, 12, 15, 11, 15, 29]
title = ["début de l'Apartheid", "Massacre de Sharpeville", "Émeutes de Soweto", "Attentat de Church Street", "état d'urgence", "légalisation des partis politiques", "libération de Mandela", "Condamnation de Mandela", "test1"]

In [ ]:
#Generate every csv from the lists above with the different dates
for d in range(0, len(datey)-1):
    print("FOR TOTAL: " + str(d) + "/" + str(len(datey)))
    name = str(datey[d]) +"_" + str(datem[d]) + ".csv"
    csvtxtwriter(name, datey[d], datem[d], dated[d], 366, -366)

In [ ]:
#create a graph from the csv files to see the occurences. We use a groupby on date with year and month only
def fromCSVtoGraph(csvname, title):
    a = pd.read_csv(csvname, delimiter='*')
    futurG = a.groupby(['date']).count().sort_values(by='date')
    ax = futurG.plot(kind='bar', title = title)
    middle = len(ax.get_xticklabels()[:]) / 2
    ax.get_xticklabels()[int(middle)].set_color("red")
    ax.legend(["Occurences"]);
    plt.savefig("Apartheid/fig/"+title+".pdf")
    return ax

In [ ]:
csv_to_read = os.listdir("Apartheid/csv/")

In [ ]:
#generate the graph from the csv for all dates
indexi = 0
for csv in csv_to_read:
    fromCSVtoGraph("Apartheid/csv/" + csv, title[indexi])
    indexi = indexi + 1